# Book Ideation and Generation using Document Review and Long Form

### Overview

The goal is to write a quality educational book in the simplest way possible. We will go through the ideation phase to set the tone and topics of the book.

##### Project Flow

1. Create knowledge bases for each chapter we want in the book.
2. Get some user inputs in regard to overall theme and style.
3. For each chapter knowledge base, run a document review where the "final review" output is a chapter outline.
4. Feed each of the document review outlines into a long form generation call.

##### Initialization

In [ ]:
import superpowered
import requests
import time
import tqdm

from IPython.display import display, Markdown

SP_BASE_URL = 'https://api.superpowered.ai/v1'
SP_API_KEY_ID = 'server_dbaf9892f0b59434ca2e751d557f979d'
SP_API_KEY_SECRET = 'qyLsmw5B5HqZV5WxOYq8Atoe2E0HjaEtaMJjrxOKf8M'
API_AUTH = (SP_API_KEY_ID, SP_API_KEY_SECRET)

knowledge_base_ids = [
    '08844d05-4e09-479d-9770-6faebccfceb6',  # STOICISM
    'a0fd538c-06ec-43ae-849b-f9570caca2c6',  # EPICUREANISM
    '87401ada-dfcb-4e3a-afbc-8fe9ead62bb4',  # MINIMALISM
    '6fde0338-0856-477b-bfa0-439432dfbcd1',  # HEDONISM
    'a78019b4-c498-412b-9daf-737698fab4c5',  # EXISTENTIALISM
    '740b2668-ecab-4372-9bc4-6a01a4f88560',  # NIHILISM
    '183cee2f-2053-480b-8e86-9377489208c5',  # PRAGMATISM
    '0e75e305-b314-40a3-a4fb-7f357bda0c10',  # IDEALISM
]


##### User input describing goal for the book

In [ ]:
BOOK_TITLE = 'Living a Good Life in the Age of AI and Automation'
BOOK_THEME = 'The general theme of this book is to look at what it means to live a good life in the context of stoicism, epicureanism, minimalism, hedonism, existentialism, nihilism, pragmatism, and idealism. Each chapter will cover one of these 8 philosophical schools of thought.'
BOOK_OVERALL_STYLE_GUIDANCE = 'The style should be widely accessible to the general public with a focus on how each of the philosophies are relevant in the modern age of AI and automation. It is important to use a variable vocabulary and make sure that nothing is repetitive and dry. Be fun and present ideas with examples. Avoid bland closing statements like "in summary", "in conclusion", etc. The book should be a fun and engaging read.'
BOOK_CHAPTER_STYLE_GUIDANCE = 'Each chapter should start with a general overview of the philosophy in question and explain how the concepts are relevant in the modern age of AI and automation. Give examples of what living by this philosophy would look like in scenarios like: "remote work and lack of social interacation", "complete automation with no jobs", "human population expantion", "virtual reality", "augmented reality", "cell phones", "immersive video games", etc.'

##### Helper functions that will run document reviews and long form plus stream results

In [ ]:
def run_and_display_review(kb: dict, ar_instructions: str, fr_instructions: str):
    url = f'{SP_BASE_URL}/knowledge_bases/{kb["id"]}/review'
    payload = {
        'active_reading_instructions': ar_instructions,
        'final_review_instructions': fr_instructions,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    final_review_display = display(Markdown('Review pending...'), display_id=True)
    progress_bar = tqdm.tqdm(total=1)
    progress_bar.set_description(f'Active Reading Progress for {kb["title"]}')

    # keep request job until it's completed (or failed)
    while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
        resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
        progress_bar.n = resp.json()['response']['active_reading_progress_pct']
        if progress_bar.n >= 1:
            final_review_display.update(Markdown(resp.json()["response"]["final_review"]))
        progress_bar.refresh()

    progress_bar.close()
    return resp.json()['response']['final_review']


def submit_long_form_job(kb: dict, prompt: str):
    url = f'{SP_BASE_URL}/long_form'
    payload = {
        'knowledge_base_ids': [kb['id']],
        'model': 'mistral-medium',
        'response_length': 'long',
        'prompt': prompt,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    return resp.json()['id']


##### Loop through each knowledge base and run the document review

In [ ]:
active_reading_instructions = f"""\
You are reading through a knowledge base on {{philosophy}}. Please extract information that is relevant to living a good life.

The infomration that is extracted will be used to write a chapter about {{philosophy}} for a book with the following theme: {BOOK_THEME}

Your output should be a list of key points and examples that could be relevant to the modern age of AI and automation. The information you are reading through might not directly mention AI and automation so it's up to you to decide if the information is relevant or not.
"""

final_review_instructions = f"""\
Given the list key points and examples from the active reading, please generate a chapter outline. The book must follow these style guidelines:

1. {BOOK_OVERALL_STYLE_GUIDANCE}
2. {BOOK_CHAPTER_STYLE_GUIDANCE}

The chapter outline should contain a list of detailed section descriptions. Please try to keep the number of sections between 5 and 7
"""

outlines = []
for kb_id in knowledge_base_ids:
    # get the knowledge base object
    kb_url = f'{SP_BASE_URL}/knowledge_bases/{kb_id}'
    resp = requests.get(kb_url, auth=API_AUTH)
    kb = resp.json()

    outline = run_and_display_review(kb, active_reading_instructions.format(philosophy=kb['title']), final_review_instructions)
    outlines.append((kb, outline))

In [ ]:
chapter_ids = []

for kb, outline in outlines:
    prompt = f"""\
    You have been assigned to write a chapter in a book titled "{BOOK_TITLE}" about {kb['title']}. Please follow this outline when writing the chapter:

    {outline}

    {BOOK_OVERALL_STYLE_GUIDANCE}
    {BOOK_CHAPTER_STYLE_GUIDANCE}

    DO NOT EVER USE CLOSINGS LIKE "in conclusion", "in summary", "ultimately", etc. They are boring and you should have more pride in your output.
    """
    print('STARTING CHAPTER:', kb['title'])
    chapter_job_id = submit_long_form_job(kb, prompt)
    chapter_ids.append(chapter_job_id)
    